## KBS 뉴스 제목, 작성날짜, url 가져오기

### selenium

In [ ]:
import sys, os
from selenium import webdriver
from pandas import DataFrame
from bs4 import BeautifulSoup
from datetime import datetime
import time
from tqdm import tqdm

In [ ]:
date = str(datetime.now())
date = date[:date.rfind(':')].replace(' ', '_')
date = date.replace(':','시') + '분'

sleep_sec = 0.5

In [ ]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver", options=options)
driver.execute_script('return navigator.userAgent')

In [ ]:
query = input('검색할 키워드  : ')

driver = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver", options=options)
driver.get('https://news.kbs.co.kr/search/search.do?query={}#1'.format(query))
driver.implicitly_wait(sleep_sec)

html = driver.page_source
soup = BeautifulSoup(html)

table = soup.find('div', {'class' : 'section result'})
news_num = table.find('p').find('em').text.replace(",", "")
print(int(news_num))

In [ ]:
news_dict = {}
idx = 0

options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36")
options.add_argument("--blink-settings=imagesEnabled=false")

driver = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver", options=options)

for i in tqdm(range(1, int(news_num)//10 + 1)): # 페이지 수
    driver.get('https://news.kbs.co.kr/search/search.do?query={}#{}'.format(query, i))
    driver.refresh()
    soup = BeautifulSoup(driver.page_source)

    table = soup.find('div', {'class' : 'section result'})

    table = driver.find_element_by_xpath('//ul[contains(@id, "searchList")]')
    li_list = table.find_elements_by_xpath('./li')

    for li in li_list:
        news_url = li.find_element_by_xpath('./a').get_attribute('href')
        news_title = li.find_element_by_xpath('./a/*/em[@class="tit"]').text
        news_time = li.find_element_by_xpath('./a/*/span[@class="time"]').text
        
        news_dict[idx] = {'title' : news_title,
                        'url' : news_url,
                        'time' : news_time
                        }
        idx += 1

driver.close()

In [ ]:
news_df = DataFrame(news_dict).T

folder_path = os.getcwd()
cvs_file_name = 'KBS_{}_{}개_{}.csv'.format(query, news_num, date)

news_df.to_csv(cvs_file_name)

print('데이터 저장 완료 | 경로 : {}\\{}\n'.format(folder_path, cvs_file_name))

news_df.head()

## KBS 뉴스 내용 가져오기

### selenium

In [ ]:
import sys, os
from selenium import webdriver
from selenium.webdriver.common.by import By
from pandas import DataFrame
from bs4 import BeautifulSoup
import time
import pandas as pd

In [ ]:
news_df = pd.read_excel("/Users/yeonsun/Documents/capstone-2022-29/crawling/한국경제_코로나_93542개_2022-03-28_03시46분_기사내용미포함.xlsx")

sleep_sec = 0.5

In [ ]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver", options=options)
driver.execute_script('return navigator.userAgent')

In [ ]:
contents = []

options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36")
options.add_argument("--blink-settings=imagesEnabled=false")

driver = webdriver.Chrome(executable_path="/opt/homebrew/bin/chromedriver", options=options)

for idx in tqdm(range(0, news_df.shape[0] + 1)):
    article_url = news_df['url'][idx]
    driver.get(article_url)
    time.sleep(sleep_sec)

    content = driver.find_element(by=By.XPATH, value='//*[@id="cont_newstext"]').text
    contents.append(content)

driver.close()

In [ ]:
idx = 0
for i in range(0, news_df.shape[0] + 1):
    news_df['content'][i] = contents[idx]
    idx += 1

In [ ]:
folder_path = os.getcwd()
cvs_file_name = 'KBS_{}_{}개_{}.csv'.format(query, news_num, date)

news_df.to_csv(cvs_file_name)

print('데이터 저장 완료 | 경로 : {}\\{}\n'.format(folder_path, cvs_file_name))

news_df.head()